In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options 
import subprocess
import os
from selenium.webdriver.support.select import Select

from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.alert import Alert
from datetime import date, timedelta


In [5]:
#드라이버로 크롬 시크릿창 열기
def get_driver() :

    # # ChromeDriverManager를 통해 ChromeDriver 설치
    # driver_path = ChromeDriverManager().install()

    # # 설치된 ChromeDriver의 버전 확인
    # version_result = subprocess.run([driver_path, "--version"], capture_output=True, text=True)
    # print(version_result.stdout)


    options = webdriver.ChromeOptions()
    # 브라우저가 백그라운드에서 실행되며, 보통 로컬환경에서는 크롤링 진행과정을 보면서 개발하는게 좋기 때문에 주석처리합니다.
    # options.add_argument("headless")

    # 지정한 user-agent로 설정
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664 Safari/537.36")

    # 크롬 화면 크기를 설정(but 반응형 사이트에서는 html요소가 달라질 수 있음)
    options.add_argument("window-size=1440x900")

    # 브라우저가 백그라운드에서 실행되며, 보통 로컬환경에서는 크롤링 진행과정을 보면서 개발하는게 좋기 때문에 주석처리합니다.
    # options.add_argument("headless")

    # ChromeDriverManager를 통해 현재 OS에 설치된 크롬브라우저 버전을 다운로드하고, 설정한 옵션을 적용합니다.
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

    url = "https://www.hometax.go.kr/websquare/websquare.wq?w2xPath=/ui/pp/index_pp.xml&tmIdx=3800000000"
    driver.get(url)
    #화면로딩대기
    driver.implicitly_wait(60)
    return driver

def login_redirect(driver:webdriver):
    #로그인버튼 클릭
    driver.find_element(By.ID, 'group1544').click()

    #화면로딩대기
    driver.implicitly_wait(60)

    #iframe 내부요소선택
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
    )
    #방화벽/보안프로그램 이용 동의체크
    driver.find_element(By.CLASS_NAME, "w2checkbox_item_0").click()
    driver.find_element(By.CLASS_NAME, "w2checkbox_item_1").click()
    driver.find_element(By.ID, "btnApply").click()
    #alert창 확인
    driver.switch_to.alert.accept()
    #화면로딩대기
    time.sleep(1)
    #메인컨텐츠 재선택
    driver.switch_to.default_content()
    # iframe이 로드될 때까지 최대 10초간 기다립니다.
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
    )
    #간편인증 선택
    driver.find_element(By.ID, "anchor14").click()
    #화면로딩대기
    time.sleep(0.5)
    #간편인증 버튼클릭
    driver.find_element(By.ID, "anchor23").click()

    WebDriverWait(driver, 20).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "UTECMADA02_iframe"))
    )
    WebDriverWait(driver, 20).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "simple_iframeView"))
    )


def login_easy(driver:webdriver):

    
    #간편인증 선택된 항목 클릭
    text = "네이버"
    # XPath를 사용하여 조건에 맞는 label 선택
    # 이 XPath는 'label' 내부에 클래스가 'label-nm'인 'span'이 있고, 그 안에 텍스트가 '원하는 텍스트'인 'p' 태그가 있는 요소를 찾습니다.
    label_to_click = driver.find_element(By.XPATH, f"//label[.//span[contains(@class, 'label-nm')]//p[text()='{text}']]")

    # 찾은 label 클릭
    label_to_click.click()

    #통신사패스 빼고는 본인인증란 동일
    name = "yourname"
    birth = "yyyymmdd"
    phone1 = "010"
    phone2 = "00000000"

    try:
        # 드롭다운 요소를 찾습니다.
        dropdown_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'select[data-id="oacx_phone1"]'))
        )
        select = Select(dropdown_element)
        select.select_by_value(phone1)  # phone1은 해당 드롭다운에서 선택하려는 옵션의 value입니다.

        # 이름 입력 필드
        name_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[data-id="oacx_name"]'))
        )
        name_field.send_keys(name)

        # 생년월일 입력 필드
        birth_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[data-id="oacx_birth"]'))
        )
        birth_field.send_keys(birth)

        # 추가 전화번호 입력 필드
        phone2_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[data-id="oacx_phone2"]'))
        )
        phone2_field.send_keys(phone2)

    except expt.NoSuchElementException:
        print("요청한 요소를 찾을 수 없습니다.")
    except expt.TimeoutException:
        print("요청한 요소가 지정된 시간 내에 나타나지 않았습니다.")
    except Exception as e:
        print(f"예외가 발생했습니다: {e}")

    #서비스이용동의
    driver.find_element(By.ID, "totalAgree").click()
    time.sleep(0.5)

    #인증요청 버튼 클릭
    driver.find_element(By.ID, "oacx-request-btn-pc").click()

    # try:
    #     # 'alertArea' 클래스가 보이는지 10초 동안 기다립니다.
    #     WebDriverWait(driver, 3).until(
    #         EC.visibility_of_element_located((By.CLASS_NAME, "alertArea"))
    #     )
    #     # 'alertArea'가 보이면 사용자 정의 예외를 발생시킵니다.
    #     raise Exception("사용자 정보 불일치")
    # except expt.TimeoutException:
    #     # 'alertArea'가 10초 동안 보이지 않으면, 정상적인 처리를 계속합니다.
    #     print("No alertArea present. Continuing with normal processing.")

    #인증확인했다고치고... 대기
    time.sleep(15)

    #인증확인 버튼 클릭
    driver.find_element(By.CSS_SELECTOR, ".btnArea button:nth-of-type(2)").click()

    #화면로딩대기
    time.sleep(3)


#세무대리수임동의
def page_locate2(driver:webdriver):
    #메인컨텐츠 재선택
    driver.switch_to.default_content()
    
    #세무대리 납세관리 클릭
    driver.find_element(By.ID, "hdGrp920").click()

    #화면로딩대기_redirect
    driver.implicitly_wait(60)

    #메인컨텐츠 재선택
    driver.switch_to.default_content()
    # iframe이 로드될 때까지 최대 10초간 기다립니다.
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))    
    )

    #세무대리 수임동의 클릭
    driver.find_element(By.ID, "a_a_4806020000").click()



#세무대리 / 납세관리
def page_locate(driver:webdriver):
    #메인컨텐츠 재선택
    driver.switch_to.default_content()

    #세무대리 납세관리 클릭
    driver.find_element(By.ID, "hdGrp920").click()

    #화면로딩대기_redirect
    driver.implicitly_wait(60)

    #메인컨텐츠 재선택
    driver.switch_to.default_content()
    # iframe이 로드될 때까지 최대 10초간 기다립니다.
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
    )
    # 납세관리인 설정/관리/변경 클릭
    driver.find_element(By.ID, "group01476").click()



In [ ]:
# 최대 대기 시간을 초 단위로 설정
LONG_WAIT_TIME = 10  # 예: 10초
DEFAULT_WAIT_TIME = 5  # 예: 5초

def income_locate(driver):
    try:
        driver.switch_to.default_content()
        driver.find_element(By.ID, "hdGroup917").click()#국세증명/사업자등록/세금관련신청신고
        WebDriverWait(driver, LONG_WAIT_TIME)
        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )
        driver.find_element(By.ID, "group_li_4301100000").click()#소득금액 증명
        WebDriverWait(driver, LONG_WAIT_TIME)
    except Exception as e:
        # driver.quit()
        print(f"국세증명 페이지이동 오류 발생: {str(e)}")
    return None

In [91]:
# 최대 대기 시간을 초 단위로 설정
LONG_WAIT_TIME = 10  # 예: 10초
DEFAULT_WAIT_TIME = 5  # 예: 5초

def income_process(driver):

    wait = WebDriverWait(driver, LONG_WAIT_TIME)
    try:
        
        driver.switch_to.default_content()
        driver.find_element(By.ID, "hdGroup917").click()#국세증명/사업자등록/세금관련신청신고
        WebDriverWait(driver, LONG_WAIT_TIME)
        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )
        driver.find_element(By.ID, "group_li_4301100000").click()#소득금액 증명
        WebDriverWait(driver, LONG_WAIT_TIME)

        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )
        time.sleep(3)
        #발급유형
        radio1 = wait.until(EC.presence_of_element_located((By.ID, "rdoEnglCvaAplnYn_input_0")))
        radio1.click()
        #과세기간
        current_date = date.today()
        year = current_date.replace(year=current_date.year - 1)
        year = year.strftime('%Y')

        print(f"year {year}")

        year1 = wait.until(EC.presence_of_element_located((By.ID, "sbTxnrmStrtYr")))
        year2 = wait.until(EC.presence_of_element_located((By.ID, "sbTxnrmEndYr")))
        select1 = Select(year1)
        select2 = Select(year2)
        select1.select_by_visible_text(year)
        select2.select_by_visible_text(year)

        #소득발생처
        driver.find_element(By.ID, "rdoIncOcplOpYn_input_0").click()
        #사용용도
        dropdown_element = wait.until(EC.presence_of_element_located((By.ID, "sbCvaDcumUseUsgCd")))
        select = Select(dropdown_element)
        select.select_by_visible_text("기타")
        #제출처
        dropdown_element2 = wait.until(EC.presence_of_element_located((By.ID, "sbCvaDcumSbmsOrgnClCd")))
        select = Select(dropdown_element2)
        select.select_by_visible_text("기타")

        #신청하기 버튼 클릭
        trigger = wait.until(EC.presence_of_element_located((By.ID, "trigger167")))
        trigger.click()

        #alret확인
        alert = WebDriverWait(driver, 10).until(EC.alert_is_present())
        alert.accept()

        alert2 = WebDriverWait(driver, 10).until(EC.alert_is_present())
        alert2.accept()

        #페이지이동 대기
        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )

        # 현재 창의 핸들을 저장
        original_window = driver.current_window_handle
        print("Current page title is:", driver.title)

        #출력버튼 클릭
        output_link = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//span[@class='w2grid_span']//a[text()='출력']")
        ))
        output_link.click()
        alert3= WebDriverWait(driver, 10).until(EC.alert_is_present())
        alert3.accept()

        # 새 창이 열릴 때까지 대기
        wait.until(EC.new_window_is_opened(driver.window_handles))

        # 모든 창 핸들을 가져와 새 창으로 전환
        all_windows = driver.window_handles
        for window_handle in all_windows:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break

        print("Change page title is:", driver.title)

        try:
            # 최대 대기 시간 설정
            wait = WebDriverWait(driver, 10)

            # PDF 저장 버튼이 나타날 때까지 대기
            pdf_btn = wait.until(
                EC.presence_of_element_located((By.XPATH, "//button[@title='PDF 저장']"))
            )

            # JavaScript를 사용하여 요소의 'disabled' 속성을 제거하고 스타일을 변경
            driver.execute_script('arguments[0].removeAttribute("disabled")', pdf_btn)
            driver.execute_script("arguments[0].style.display='block'", pdf_btn)

            time.sleep(2)
            print("대기")

            # JavaScript를 사용하여 요소 클릭
            driver.execute_script("arguments[0].click();", pdf_btn)

            print("PDF 저장 버튼 클릭 성공")

        except Exception as e:
            print("저장실패:", e)

        # 두 번째 창 닫기
        driver.close()

        # 원래의 첫 번째 창으로 다시 전환
        driver.switch_to.window(original_window)
        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )
    except Exception as e:
        print(f"소득금액신청서 내역조회 오류 발생: {str(e)}")

In [88]:
pdf_btn = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//button[@title='PDF 저장']"))
)
driver.execute_script(
        'arguments[0].removeAttribute("disabled")', pdf_btn)
driver.execute_script("arguments[0].style.display='block'", pdf_btn)

# JavaScript를 사용하여 요소 클릭
driver.execute_script("arguments[0].click();", pdf_btn)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.78)
Stacktrace:
0   chromedriver                        0x0000000102816510 chromedriver + 4302096
1   chromedriver                        0x000000010280ee58 chromedriver + 4271704
2   chromedriver                        0x000000010244019c chromedriver + 278940
3   chromedriver                        0x000000010241b474 chromedriver + 128116
4   chromedriver                        0x00000001024a7394 chromedriver + 701332
5   chromedriver                        0x00000001024ba4e4 chromedriver + 779492
6   chromedriver                        0x0000000102477004 chromedriver + 503812
7   chromedriver                        0x00000001024779ec chromedriver + 506348
8   chromedriver                        0x00000001027de558 chromedriver + 4072792
9   chromedriver                        0x00000001027e3004 chromedriver + 4091908
10  chromedriver   

In [84]:
print("Current page title is:", driver.title)
original_window = driver.current_window_handle
all_windows = driver.window_handles
for window_handle in all_windows:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break
print("change page title is:", driver.title)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.78)
Stacktrace:
0   chromedriver                        0x00000001011ce510 chromedriver + 4302096
1   chromedriver                        0x00000001011c6e58 chromedriver + 4271704
2   chromedriver                        0x0000000100df819c chromedriver + 278940
3   chromedriver                        0x0000000100dd3474 chromedriver + 128116
4   chromedriver                        0x0000000100e5f394 chromedriver + 701332
5   chromedriver                        0x0000000100e724e4 chromedriver + 779492
6   chromedriver                        0x0000000100e2f004 chromedriver + 503812
7   chromedriver                        0x0000000100e2f9ec chromedriver + 506348
8   chromedriver                        0x0000000101196558 chromedriver + 4072792
9   chromedriver                        0x000000010119b004 chromedriver + 4091908
10  chromedriver   

In [79]:
pdf_btn = WebDriverWait(driver, 2).until(
    EC.presence_of_element_located((By.XPATH, "//button[@title='PDF 저장']"))
)
driver.execute_script(
        'arguments[0].removeAttribute("disabled")', pdf_btn)
driver.execute_script("arguments[0].style.display='block'", pdf_btn)

# JavaScript를 사용하여 요소 클릭
driver.execute_script("arguments[0].click();", pdf_btn)

In [83]:
income_process(driver)
# 두 번째 창 닫기
# print("Current page title is:", driver.title)
# driver.close()

# # 원래의 첫 번째 창으로 다시 전환
# driver.switch_to.window(original_window)
# driver.switch_to.default_content()
# WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
#     EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
# )

소득금액신청서 내역조회 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.78)
Stacktrace:
0   chromedriver                        0x00000001011ce510 chromedriver + 4302096
1   chromedriver                        0x00000001011c6e58 chromedriver + 4271704
2   chromedriver                        0x0000000100df819c chromedriver + 278940
3   chromedriver                        0x0000000100dd3474 chromedriver + 128116
4   chromedriver                        0x0000000100e5f394 chromedriver + 701332
5   chromedriver                        0x0000000100e724e4 chromedriver + 779492
6   chromedriver                        0x0000000100e2f004 chromedriver + 503812
7   chromedriver                        0x0000000100e2f9ec chromedriver + 506348
8   chromedriver                        0x0000000101196558 chromedriver + 4072792
9   chromedriver                        0x000000010119b004 chromedriver + 4091908
10  chromedriver      

In [92]:
def run():
    global driver
    driver = get_driver()
    login_redirect(driver)
    login_easy(driver)
    # income_locate(driver)
    income_process(driver)

    # driver.quit();

run()

year 2023
Current page title is: 국세청 홈택스 - 국세증명ㆍ사업자등록ㆍ세금관련 신청/신고 > 즉시발급 증명 > 소득금액증명
Change page title is: Report
대기
PDF 저장 버튼 클릭 성공


In [14]:

# 최대 대기 시간을 초 단위로 설정
LONG_WAIT_TIME = 10  # 예: 10초
DEFAULT_WAIT_TIME = 3  # 예: 5초

def tax_return_locate(driver):
    try:
        driver.switch_to.default_content()
        driver.find_element(By.ID, "hdGroup917").click()#국세증명/사업자등록/세금관련신청신고
        WebDriverWait(driver, LONG_WAIT_TIME)
        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )
        driver.find_element(By.ID, "group_li_4301100000").click()#소득금액 증명
        WebDriverWait(driver, LONG_WAIT_TIME)
    except Exception as e:
        # driver.quit()
        print(f"국세증명 페이지이동 오류 발생: {str(e)}")
    return None

def tax_return_process(driver):
    try:
        driver.switch_to.default_content()
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
        )
        wait = WebDriverWait(driver, DEFAULT_WAIT_TIME)
        #발급유형
        driver.find_element(By.ID, "rdoEnglCvaAplnYn_input_0").click()
        #과세기간

        #소득발행처

        #사용용도

        #제출처
        dropdown_element = wait.until(EC.presence_of_element_located((By.ID, "selectbox_itrfCd")))
        select = Select(dropdown_element)
        select.select_by_visible_text("종합소득세")

        res_num = wait.until(EC.visibility_of_element_located((By.ID, "input_txprRgtNo")))
        res_num.clear()
        res_num.send_keys("00112212345678")

        driver.find_element(By.ID, "ntplInfpYn_input_1").click()
        driver.find_element(By.ID, "radioWrtnRtnYn_input_0").click()

        current_date = date.today()
        for i in range(2, 6):
            start_of_year = current_date.replace(year=current_date.year - i, month=1, day=1)
            end_of_year = current_date.replace(year=current_date.year - i, month=12, day=31)
            start_date = start_of_year.strftime('%Y-%m-%d')
            end_date = end_of_year.strftime('%Y-%m-%d')

            print(f"first {start_date}, last: {end_date}")

            date1 = wait.until(EC.visibility_of_element_located((By.ID, "rtnDtSrt_input")))
            date2 = wait.until(EC.visibility_of_element_located((By.ID, "rtnDtEnd_input")))

            date1.clear()
            date1.send_keys(start_date)
            date2.clear()
            date2.send_keys(end_date)

            time.sleep(0.5)
            driver.find_element(By.ID, "trigger71").click()
            alert_text = ''
            try:
                alert = WebDriverWait(driver, 10).until(EC.alert_is_present())
                alert_text = alert.text
                alert.accept()
            except TimeoutException:
                print("No alert present after click.")

            # 알림 창의 내용에 따라 분기 처리
            if "확인" in alert_text:
                print("주민등록번호 오류")
                break
                # 필요한 처리 수행
            elif "없습니다" in alert_text:
                print("데이터없음")
                continue
            
            wait = WebDriverWait(driver,3)
            try:
                print("button wait1")
                button = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#ttirnam101DVOListDes_cell_0_10 > span"))
                )
                # button = driver.find_element(By.ID, "rdoAnsw3_input_0")
                print("button wait2")
                button.click()
                print("결과내용")

                print("Current page title is:", driver.title)
                time.sleep(3)
                original_window = driver.current_window_handle
                all_windows = driver.window_handles
                for window_handle in all_windows:
                    if window_handle != original_window:
                        driver.switch_to.window(window_handle)
                        break
                print("change page title is:", driver.title)
                
                driver.switch_to.default_content()

                #pdf다운로드 버튼 클릭
                pdf_btn = driver.find_element(By.ID, "re_pdfc1341b32b6cb54baba1e76ef8726d1085rpt7")
                print("pdfbtn",pdf_btn)
                pdf_btn.click()
                time.sleep(2)

                element1 = driver.execute_script("return document.querySelector('[aria-label=\"과세연월\"]');")
                # 바로 다음 'span' 요소를 찾습니다. JavaScript에서 nextElementSibling을 사용합니다.
                next_span = driver.execute_script("return arguments[0].nextElementSibling;", element1)

                # 바로 다음 'span' 요소의 'aria-label' 속성을 출력합니다.
                if next_span:
                    next_span_aria_label = next_span.get_attribute('aria-label')
                    print("다음 span의 aria-label 내용:", next_span_aria_label)

                element2 = driver.execute_script("""
                return Array.from(document.querySelectorAll('span[aria-label]')).find(element2 => 
                    /종합소득세.*과세표준/s.test(element2.getAttribute('aria-label')));
                """)
                # 바로 다음 'span' 요소를 찾습니다. JavaScript에서 nextElementSibling을 사용합니다.
                next_span2 = driver.execute_script("return arguments[0].nextElementSibling;", element2)

                # 바로 다음 'span' 요소의 'aria-label' 속성을 출력합니다.
                if next_span2:
                    next_span_aria_label2 = next_span2.get_attribute('aria-label')
                    print("다음 span의 aria-label 내용:", next_span_aria_label2)

                # 두 번째 창 닫기
                driver.close()

                # 원래의 첫 번째 창으로 다시 전환
                driver.switch_to.window(original_window)
                driver.switch_to.default_content()
                WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
                )

            except NoSuchElementException:
                print("결과없음")
            except TimeoutException:
                print("버튼을 찾는데 시간이 너무 오래 걸립니다.")

    except Exception as e:
        print(f"전자신고 내역조회 오류 발생: {str(e)}")
    # finally:
        # driver.quit()



In [15]:
def run():
    global driver
    driver = get_driver()
    login_redirect(driver)
    login_easy(driver)
    tax_return_locate(driver)
    tax_return_process(driver)

    # driver.quit();

run()

first 2022-01-01, last: 2022-12-31
데이터없음
first 2021-01-01, last: 2021-12-31
button wait1
button wait2
결과내용
Current page title is: 국세청 홈택스 - 세금신고 > 신고서 조회/삭제/부속서류 > 전자신고 결과 조회
change page title is: Report
결과없음
first 2020-01-01, last: 2020-12-31
전자신고 내역조회 오류 발생: Message: 
Stacktrace:
0   chromedriver                        0x000000010056e4e8 chromedriver + 4302056
1   chromedriver                        0x0000000100566e30 chromedriver + 4271664
2   chromedriver                        0x000000010019819c chromedriver + 278940
3   chromedriver                        0x00000001001da2c4 chromedriver + 549572
4   chromedriver                        0x0000000100212c5c chromedriver + 781404
5   chromedriver                        0x00000001001cf004 chromedriver + 503812
6   chromedriver                        0x00000001001cf9ec chromedriver + 506348
7   chromedriver                        0x0000000100536530 chromedriver + 4072752
8   chromedriver                        0x000000010053afdc chromed

In [10]:
tax_return_process(driver)

전자신고 내역조회 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.76)
Stacktrace:
0   chromedriver                        0x000000010243e4e8 chromedriver + 4302056
1   chromedriver                        0x0000000102436e30 chromedriver + 4271664
2   chromedriver                        0x000000010206819c chromedriver + 278940
3   chromedriver                        0x0000000102043474 chromedriver + 128116
4   chromedriver                        0x00000001020cf394 chromedriver + 701332
5   chromedriver                        0x00000001020e24e4 chromedriver + 779492
6   chromedriver                        0x000000010209f004 chromedriver + 503812
7   chromedriver                        0x000000010209f9ec chromedriver + 506348
8   chromedriver                        0x0000000102406530 chromedriver + 4072752
9   chromedriver                        0x000000010240afdc chromedriver + 4091868
10  chromedriver         

In [103]:
print("first 2222:", driver.title)
driver.switch_to.default_content()
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "txppIframe"))
)
print("button wait1")
button = WebDriverWait(driver, 2).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#ttirnam101DVOListDes_cell_0_10 > span"))
)
# button = driver.find_element(By.ID, "rdoAnsw3_input_0")
print("button wait2")
button.click()
print("결과내용")

original_window = driver.current_window_handle
all_windows = driver.window_handles
for window_handle in all_windows:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

print("change page title is:", driver.title)

driver.switch_to.default_content()
WebDriverWait(driver, 3).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "re_iframe3ce4cab53b595d47eea503cb44f47fc79bpt10"))
)
# 모든 과세연월 요소가 존재할 때까지 기다립니다.
target_elements = WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.XPATH, "//span[@aria-label='과세연월']"))
)

for element in target_elements:
    print(element.text)

# 두 번째 창 닫기
driver.close()

# 원래의 첫 번째 창으로 다시 전환
driver.switch_to.window(original_window)
print("change 2222:", driver.title)


first 2222: 국세청 홈택스 - 세금신고 > 신고서 조회/삭제/부속서류 > 전자신고 결과 조회
button wait1
button wait2
결과내용
change page title is: Report


TypeError: WebDriverWait.until() got an unexpected keyword argument 'target_elements'

In [192]:
# 현재 창의 타이틀 출력
print("Current page title is:", driver.title)
original_window = driver.current_window_handle
all_windows = driver.window_handles
for window_handle in all_windows:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break
print("change page title is:", driver.title)

# 두 번째 창 닫기
# driver.close()

# 원래의 첫 번째 창으로 다시 전환
# driver.switch_to.window(original_window)
# print("change 2222:", driver.title)

# driver.switch_to.default_content()
# WebDriverWait(driver, 3).until(
#     EC.frame_to_be_available_and_switch_to_it((By.ID, "re_iframe3ce4cab53b595d47eea503cb44f47fc79bpt10"))
# )

# # iframe 내부의 첫 번째 요소를 찾고, 클래스 이름과 ID를 출력
# element_html = first_element.get_attribute('innerHTML')  # 요소의 외부 HTML 가져오기
# print("First element HTML content:\n", element_html)

# print("First element class name:", class_name)
# print("First element ID:", element_id)

# WebDriverWait(driver, 5).until(
#     target_elements = driver.find_elements(By.XPATH, "//span[@aria-label='과세연월']")
# )

Current page title is: 국세청 홈택스 - 세금신고 > 신고서 조회/삭제/부속서류 > 전자신고 결과 조회
change page title is: 국세청 홈택스 - 세금신고 > 신고서 조회/삭제/부속서류 > 전자신고 결과 조회


In [184]:
# 현재 창의 타이틀 출력
print("Current page title is:", driver.title)
driver.switch_to.default_content()
target_div = driver.find_element(By.ID, "re_paintDivc0c90f76db4cc4a4e94b97f5d95f8fb9bpt10")

# 해당 요소의 내용, 속성 등을 조작하거나 검사할 수 있습니다.
print(target_div.get_attribute('innerHTML'))  # 요소의 내부 HTML 출력
# print("First element HTML content:\n", element_html)
# driver.switch_to.window(original_window)
# print("change 2222:", driver.title)
# driver.close()

Current page title is: 국세청 홈택스 - 세금신고 > 신고서 조회/삭제/부속서류 > 전자신고 결과 조회


KeyboardInterrupt: 

In [34]:
print("Current page title is:", driver.title)
driver.switch_to.default_content()
# element = driver.find_element(By.XPATH, "//span[@aria-label='과세연월']")
# print("과세연월 내용:", element.text)  # 요소의 텍스트 내용을 출력합니다.

 # JavaScript를 실행하여 'aria-label="과세연월"'이라는 요소를 찾습니다.
element = driver.execute_script("return document.querySelector('[aria-label=\"과세연월\"]');")

# 바로 다음 'span' 요소를 찾습니다. JavaScript에서 nextElementSibling을 사용합니다.
next_span = driver.execute_script("return arguments[0].nextElementSibling;", element)

# 바로 다음 'span' 요소의 'aria-label' 속성을 출력합니다.
if next_span:
    next_span_aria_label = next_span.get_attribute('aria-label')
    print("다음 span의 aria-label 내용:", next_span_aria_label)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.78)
Stacktrace:
0   chromedriver                        0x0000000104b06510 chromedriver + 4302096
1   chromedriver                        0x0000000104afee58 chromedriver + 4271704
2   chromedriver                        0x000000010473019c chromedriver + 278940
3   chromedriver                        0x000000010470b474 chromedriver + 128116
4   chromedriver                        0x0000000104797394 chromedriver + 701332
5   chromedriver                        0x00000001047aa4e4 chromedriver + 779492
6   chromedriver                        0x0000000104767004 chromedriver + 503812
7   chromedriver                        0x00000001047679ec chromedriver + 506348
8   chromedriver                        0x0000000104ace558 chromedriver + 4072792
9   chromedriver                        0x0000000104ad3004 chromedriver + 4091908
10  chromedriver   

In [173]:
print("Current page title is:", driver.title)
driver.switch_to.default_content()

element1 = driver.execute_script("return document.querySelector('[aria-label=\"과세연월\"]');")
# 바로 다음 'span' 요소를 찾습니다. JavaScript에서 nextElementSibling을 사용합니다.
next_span = driver.execute_script("return arguments[0].nextElementSibling;", element1)

# 바로 다음 'span' 요소의 'aria-label' 속성을 출력합니다.
if next_span:
    next_span_aria_label = next_span.get_attribute('aria-label')
    print("다음 span의 aria-label 내용:", next_span_aria_label)

element2 = driver.execute_script("""
return Array.from(document.querySelectorAll('span[aria-label]')).find(element2 => 
    /종합소득세.*과세표준/s.test(element2.getAttribute('aria-label')));
""")
# 바로 다음 'span' 요소를 찾습니다. JavaScript에서 nextElementSibling을 사용합니다.
next_span2 = driver.execute_script("return arguments[0].nextElementSibling;", element2)

# 바로 다음 'span' 요소의 'aria-label' 속성을 출력합니다.
if next_span2:
    next_span_aria_label2 = next_span2.get_attribute('aria-label')
    print("다음 span의 aria-label 내용:", next_span_aria_label2)

Current page title is: Report
다음 span의 aria-label 내용: 2020년01월
다음 span의 aria-label 내용: 0
